## Introduction to Dataset Processing
#### by Carl Shan and Jen Selby

This Jupyter Notebook will share more details about how to process your data. 

Data processing is like preparing the ingredients before cooking; if you prepare them poorly (e.g., leave things half-peeled and dirty) , the meal will taste poor no matter how skillful a chef you are. 

It's similarly true in machine learning. Dataset processing can be one of the most important things you can do to get your model to perform well.

You can read more about dataset processing on the [course notes here](https://jennselby.github.io/MachineLearningCourseNotes/#data-processing).

### Hint: Use the `?` symbol

As you go through this notebook, as well as learn more about processing data in iPython, it will be helpful to know the `?` symbol.

E.g., You can try to type the following into Python

```python
import sklearn

sklearn?
```

Typing the `?` symbol after a function, module or variable will bring up the documentation of that bit of code, assuming it exists. It'll tell you more about the variable, function or module.


## Dataset processing: Setup

In [1]:
import pandas as pd
from sklearn import preprocessing

Download the [student performance data](http://archive.ics.uci.edu/ml/machine-learning-databases/00320/) and change the path below to wherever you put the data.

In [ ]:
student_data = pd.read_csv('../data/student/student-mat.csv', sep=';')

In [ ]:
student_data.head()

## Section 1: Converting Categorical Values to Numerical Ones

Looking at the data above, we want to convert a number of the columns from categorical to numerical. Most machine learning models deal with numbers and don't know how to model data that is in text form. As a result we need to learn how to do things such as e.g., convert the values in the `school` column to numbers.

### First, let's see what values there are in the `school` column

In [ ]:
# This shows a list of unique values and how many times they appear
student_data['school'].value_counts()

In [ ]:
# Converting values in the school column to text
# We are going to define a function that takes a single value and apply it to all the values
def convert_school(row):
    if row == 'GP':
        return 0
    elif row == 'MS':
        return 1
    else:
        return None

### Avoid for loops
Normally, we might write a for loop like the one below. But this is really slow when using Pandas.

### _Don't write loops like this_.

In [ ]:
# Here's a slow way of using the above function
%time
converted_school = []

for row in student_data['school']:
    new_value = convert_school(row)
    converted_school.append(new_value)
converted_school


# Don't do this! It's very slow.

### Instead, Use `.apply`
This will do the same thing as the for loop above, but _much_ faster. It'll apply a function to all the rows of a `DataFrame`.

In [ ]:
%time
converted_school = student_data['school'].apply(convert_school)
converted_school

Look how much faster that took!

### Or, you can use `.map()`

You can also use the `.map()` function to map certain values to other data.

For example, imagine you had a column named `'colors'` that contained the values `"red"` and `"blue"` and you wanted to convert these to the numbers `1` and `2`.

```python

mappings = {
    'red': 1,
    'blue': 2
}

data['colors_mapped'] = data['colors'].map(mappings)

```

The above will create a new column called `colors_mapped` that now has the values `1` and `2`.

### Using sklearn's built-in preprocessing module, we can do the same thing

In [ ]:
enc_school = preprocessing.LabelEncoder()
transformed_school = enc_school.fit_transform(student_data['school'])
transformed_school

### We can also use one-hot encoding if we have more than two values. We still need to encode it first, as we did above.
See example at https://stackoverflow.com/a/43589167/2159992

In [ ]:
enc_mjob = preprocessing.LabelEncoder()
encoded_mjob = enc_mjob.fit_transform(student_data['Mjob'])
encoded_mjob

In [ ]:
onehot_mjob = preprocessing.OneHotEncoder(sparse=False)
transformed_mjob = onehot_mjob.fit_transform(encoded_mjob.reshape(-1,1))
transformed_mjob

Once we've fitted the label encoder and one-hot encoder, we can use them to transform more values.

In [ ]:
onehot_mjob.transform(enc_mjob.transform(['other', 'health']).reshape(-1,1))

### What if we want to apply a transform that looks at multiple values in the row?
For instance, what if we want to create a new column with a 1 if both parents have the highest level of education measured?

In [ ]:
medu_index = student_data.columns.get_loc('Medu')
fedu_index = student_data.columns.get_loc('Fedu')

def both_parents_edu(row):
    if row[medu_index] > 3 and row[fedu_index] >= 4:
        return 1
    else:
        return 0
    
# axis 1 means that we will apply the function to each row
student_data['parents_high_edu'] = student_data.apply(both_parents_edu, axis=1)
student_data.head(10)

## Section 2: Dealing with Text Data

`pandas` has a lot of built-in modules that work with text-based data. 

`sklearn` similarly has a lot of modules for this as well.

This section gives a brief outline of the things you can try.

If you want to see a fuller list, with examples, of how `pandas` deals with text data, you can look at [the documentation here](https://pandas.pydata.org/pandas-docs/stable/text.html).

In [ ]:
#### First, I'm going to make some fake data that we can work with for the rest of this section

data = pd.DataFrame(data={'text': ['apple', '%badly,formatted,data%', 'pear']})

In [ ]:
data

### Removing or replacing data

Okay, we want to remove the `','` and `'%'` symbols from the data. How do we do so?

In [ ]:
data['text_removed'] = data['text'].str.replace(',', '')

In [ ]:
data

Nice. Now try and replace the `'%'` symbols.

In [ ]:
#### Your code here






### Checking to see if a string contains a certain value

Now, we want to see if a text contains certain values, and only get the rows that contains those values.

In [ ]:
### Again, I have to make some fake data

data = pd.DataFrame(data={'text': ['Nueva Maverick', 'San Francisco Maverick', 'Vikings']})

In [ ]:
data

Cool, what if we only wanted to get the rows that contained the word `'Maverick'`?

In [ ]:
data['text'].str.contains('Maverick')

Now we can use this `Series` of `boolean` `True` and `False` values to index into our data!

In [ ]:
condition = data['text'].str.contains('Maverick')

filtered_data = data[condition]

In [ ]:
filtered_data

### There is a whole list of other things you can do with your text data. 

**Some are listed below:**

* `str.startswith()` and `str.endswith()` - checks to see if a string starts or ends with a given argument
* `str.count()` - counts the number of appearances of a certain pattern
* `str.numeric()` - checks to see if the string is numeric (e.g., `23123` is a digit whereas `213123abc` is not)
* `str.split()` - splits the string on some deliminter and returns a dataframe of the string, split on the characters.

There's plenty more and you can see the [documentation here for more](https://pandas.pydata.org/pandas-docs/stable/text.html).


### What about transforming text data into a DataFrame?

In [2]:
raw_text = ["""This is a giant series of sentences that you want to convert into a DataFrame containing 
the raw counts for each word. There are some abbr. and some punctuations here and there that make things more complicated.
So how in the world do we turn this into something that we can build a machine learning model off of?
"""]

Okay, so we want to turn the above into a DataFrame where every column is a different word, and each entry stores the number of times that word came up.

We're going to use the `CountVectorizer` class in `sklearn`.

A more [in-depth tutorial on how to use it, and more, can be found here](https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/).

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# Initializing an empty CountVectorizer object
count_vect = CountVectorizer()

In [5]:
# Now we fit the object to our actual data
counts = count_vect.fit_transform(raw_text)

In [6]:
# This is a `sparse matrix` class. It saves our computer space.
counts

<1x45 sparse matrix of type '<class 'numpy.int64'>'
	with 45 stored elements in Compressed Sparse Row format>

In [7]:
# Let's use the `.todense()` function to turn this sparse matrix into something that can be transformed into a DataFrame
word_counts_df = pd.DataFrame(data=counts.todense())

In [8]:
word_counts_df

0   1   2   3   4   5   6   7   8   9   ...  35  36  37  38  39  40  41  \
0   1   2   1   1   1   1   1   1   1   1  ...   2   1   2   1   1   1   2   

   42  43  44  
0   1   1   1  

[1 rows x 45 columns]

Great, but what do each of the columns mean?

We can inspect the `count_vect.vocabulary_` attribute to find out.

In [9]:
count_vect.vocabulary_

{'this': 37,
 'is': 18,
 'giant': 13,
 'series': 29,
 'of': 24,
 'sentences': 28,
 'that': 33,
 'you': 44,
 'want': 40,
 'to': 38,
 'convert': 7,
 'into': 17,
 'dataframe': 9,
 'containing': 6,
 'the': 34,
 'raw': 27,
 'counts': 8,
 'for': 12,
 'each': 11,
 'word': 42,
 'there': 35,
 'are': 2,
 'some': 31,
 'abbr': 0,
 'and': 1,
 'punctuations': 26,
 'here': 14,
 'make': 21,
 'things': 36,
 'more': 23,
 'complicated': 5,
 'so': 30,
 'how': 15,
 'in': 16,
 'world': 43,
 'do': 10,
 'we': 41,
 'turn': 39,
 'something': 32,
 'can': 4,
 'build': 3,
 'machine': 20,
 'learning': 19,
 'model': 22,
 'off': 25}

Great. Now we know the words for each of the datasets

**Your challenge**: Your challenge is to write some code so that you end converting the columns in `word_counts_df` to each of the words in `count_vect.vocabulary_`.

In [11]:
# If you're successful it should look like the output below.

abbr  and  are  build  can  complicated  containing  convert  counts  \
0     1    2    1      1    1            1           1        1       1   

   dataframe ...   there  things  this  to  turn  want  we  word  world  you  
0          1 ...       2       1     2   1     1     1   2     1      1    1  

[1 rows x 45 columns]

In [ ]:
### Your code here







## Section 3: Dealing with Null values
To show you how to deal with null values, I'm going to make some simulated data of students.

In [12]:
import numpy as np
grades = np.random.choice(range(1, 13), 100) # chooses 100 random numbers between 1 - 12
num_friends_or_none = list(range(0, 20)) + [None] * 5
num_friends = np.random.choice(num_friends_or_none, 100)
new_data = pd.DataFrame(data={'Grade': grades, '# Friends': num_friends})

In [13]:
new_data.head(n=20)

Grade # Friends
0       3        18
1       4      None
2       6        14
3       2         5
4      11         8
5       3      None
6       8        17
7      12         9
8      12         9
9      12         8
10      9      None
11      2        18
12     10         9
13      9      None
14     10      None
15      3        11
16      8      None
17      9        12
18      1        14
19      2      None

#### One way to deal with null values is to drop them

In [14]:
new_data['# Friends'].dropna()

0     18
2     14
3      5
4      8
6     17
      ..
93    13
95     6
96    10
97     9
98     8
Name: # Friends, Length: 75, dtype: object

#### We can also drop any rows with nulls from the entire table.

In [15]:
new_data.dropna()

Grade # Friends
0       3        18
2       6        14
3       2         5
4      11         8
6       8        17
..    ...       ...
93      2        13
95      2         6
96      1        10
97      1         9
98      5         8

[75 rows x 2 columns]

#### Or we can replace the null values with an average

In [16]:
average_friends = new_data['# Friends'].mean()
new_data['# Friends'].fillna(average_friends)

0     18.000000
1     10.733333
2     14.000000
3      5.000000
4      8.000000
        ...    
95     6.000000
96    10.000000
97     9.000000
98     8.000000
99    10.733333
Name: # Friends, Length: 100, dtype: float64

In [17]:
new_data['# Friends'] = new_data['# Friends'].fillna(average_friends)

#### What if instead of null values, there is something else that stands for missing values?
Try the replace function.

In [18]:
grades = np.random.choice(range(1, 13), 100) # chooses 100 random numbers between 1 - 12
num_friends_or_none = list(range(0, 20)) + ["Unknown"] * 5
num_friends = np.random.choice(num_friends_or_none, 100)
unknown_data = pd.DataFrame(data={'Grade': grades, '# Friends': num_friends})
unknown_data

Grade # Friends
0       8         8
1       5        14
2      10        19
3      11         4
4       5        10
..    ...       ...
95      1   Unknown
96      3         3
97      3         8
98      5        19
99      7         1

[100 rows x 2 columns]

In [19]:
unknown_data.replace("Unknown", 10)

Grade # Friends
0       8         8
1       5        14
2      10        19
3      11         4
4       5        10
..    ...       ...
95      1        10
96      3         3
97      3         8
98      5        19
99      7         1

[100 rows x 2 columns]

## Section 4: Now let's learn how to standardize data
By that I mean to transform our data so that it has a mean of 0 and a standard deviation of 1.

Why would we want to do this?


Well often we will have strange parameter estimates on many models models if different bits of our data are in wildly different ranges.


> Many researchers have noted the importance of standardizing variables for multivariate analysis. 
>
> Otherwise, variables measured at different scales do not contribute equally to the analysis. 

>For example, in boundary detection, a variable that ranges between 0 and 100 will outweigh a variable that ranges between 0 and 1. Using these variables without standardization in effect gives the variable with the larger range a weight of 100 in the analysis. 

>Transforming the data to comparable scales can prevent this problem. Typical data standardization procedures equalize the range and/or data variability.

[Source](https://www.biomedware.com/files/documentation/Preparing_data/Why_standardize_variables.htm)



In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
scaler = StandardScaler()

In [22]:
scaler.fit_transform(new_data)

array([[-1.04321507,  1.51895171],
       [-0.72612843,  0.        ],
       [-0.09195513,  0.6828315 ],
       [-1.36030172, -1.19843896],
       [ 1.49347811, -0.57134881],
       [-1.04321507,  0.        ],
       [ 0.54221817,  1.30992166],
       [ 1.81056476, -0.36231876],
       [ 1.81056476, -0.36231876],
       [ 1.81056476, -0.57134881],
       [ 0.85930482,  0.        ],
       [-1.36030172,  1.51895171],
       [ 1.17639147, -0.36231876],
       [ 0.85930482,  0.        ],
       [ 1.17639147,  0.        ],
       [-1.04321507,  0.05574135],
       [ 0.54221817,  0.        ],
       [ 0.85930482,  0.2647714 ],
       [-1.67738837,  0.6828315 ],
       [-1.36030172,  0.        ],
       [ 0.85930482,  0.05574135],
       [ 1.49347811,  1.72798176],
       [-0.72612843, -0.98940891],
       [-1.04321507,  0.        ],
       [-0.72612843,  0.        ],
       [-0.09195513,  0.89186155],
       [ 0.54221817, -0.1532887 ],
       [ 0.22513152, -1.82552912],
       [ 0.54221817,

The above will transform the data so that all the columns have an average of 0 and a standard deviation of 1.

You can read the full documentatio for the `StandardScaler` [here](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).



## Bonus: What if we would like to split up columns?

Maybe  you have data in a column that's a mashup between multiple values.

For example, imagine if you have a column that stores values like:

`'8th Grade - 13 years old'` and `'12th grade - 17 years old'` and you want to create two columns: `grade` and `age` to store the two separate bits of data.

How do you do so?

In [23]:
# I'm going to generate some fake data here. Ignore the below>
grades = np.random.choice(range(1, 13), 100) # chooses 100 random numbers between 1 - 12
grades_and_ages = ['Grade {grade}  - {age} years old'.format(grade=grade, age=grade+6) for grade in grades]
num_friends_or_none = list(range(0, 20)) + ["Unknown"] * 5
num_friends = np.random.choice(num_friends_or_none, 100)
combined_data = pd.DataFrame(data={'Grade and Age': grades_and_ages, '# Friends': num_friends})

In [24]:
combined_data

Grade and Age # Friends
0   Grade 12  - 18 years old        12
1    Grade 4  - 10 years old         0
2     Grade 1  - 7 years old         9
3     Grade 1  - 7 years old         1
4    Grade 4  - 10 years old         4
..                       ...       ...
95   Grade 8  - 14 years old        16
96    Grade 3  - 9 years old        18
97    Grade 3  - 9 years old         3
98   Grade 9  - 15 years old         7
99  Grade 10  - 16 years old   Unknown

[100 rows x 2 columns]

Awesome, now let's split things up. We'll use the built in `.str.split()` function with the extra input `expand=True`.

The `expand=True` will convert the splitted data into a `DataFrame` instead of keeping a list of values. 

(Try taking out `expand=True` and seeing what happens.)

In [25]:
combined_data['Grade and Age'].str.split(' - ', expand=True)

0             1
0   Grade 12   18 years old
1    Grade 4   10 years old
2    Grade 1    7 years old
3    Grade 1    7 years old
4    Grade 4   10 years old
..        ...           ...
95   Grade 8   14 years old
96   Grade 3    9 years old
97   Grade 3    9 years old
98   Grade 9   15 years old
99  Grade 10   16 years old

[100 rows x 2 columns]

**Your challenge**: Write some code that does the following:

1. Removes the words 'Grade' and 'years old' from the data
2. Takes these two columns and puts them back into the original `combined_data` DataFrame.

In [57]:
def sketch(x):
    return x.replace("Grade ","").replace(" years old","")

combined_data['Grade and Age'].str.split(' - ', expand=True).applymap(sketch)

0   1
0   12   18
1    4   10
2    1    7
3    1    7
4    4   10
..  ...  ..
95   8   14
96   3    9
97   3    9
98   9   15
99  10   16

[100 rows x 2 columns]